In [1]:
## Augmentation 
# Scale a dataset to 1.5. scale factor 
# Have a dataset of red and yellow flags. Have other datasets of red flags where the red flag is larger and more zoomed in
# This is to scale the flags and update the annotations 

import os
import cv2
import numpy as np

def scale_object(image, bbox, scale_factor):
    """
    Scales the object within a bounding box by a given scale factor.
    Args:
        image (numpy array): Input image.
        bbox (tuple): Bounding box in the format (class_id, center_x, center_y, width, height) in normalized format.
        scale_factor (float): Factor by which to scale the object.
    Returns:
        numpy array: Image with the object scaled.
        tuple: Updated bounding box.
    """
    h, w, _ = image.shape
    class_id, cx, cy, bw, bh = bbox
    cx, cy, bw, bh = cx * w, cy * h, bw * w, bh * h

    # Calculate new bounding box dimensions
    new_bw = bw * scale_factor
    new_bh = bh * scale_factor

    # Crop and resize the object
    x1, y1 = int(cx - bw / 2), int(cy - bh / 2)
    x2, y2 = int(cx + bw / 2), int(cy + bh / 2)
    object_crop = image[y1:y2, x1:x2]
    object_resized = cv2.resize(object_crop, (int(new_bw), int(new_bh)))

    # Replace the object in the image
    new_x1, new_y1 = int(cx - new_bw / 2), int(cy - new_bh / 2)
    new_x2, new_y2 = new_x1 + object_resized.shape[1], new_y1 + object_resized.shape[0]

    # Ensure boundaries are within the image dimensions
    new_x1 = max(0, new_x1)
    new_y1 = max(0, new_y1)
    new_x2 = min(w, new_x2)
    new_y2 = min(h, new_y2)

    # Paste the resized object back into the image
    image[new_y1:new_y2, new_x1:new_x2] = object_resized

    # Update bounding box to normalized format
    new_cx = (new_x1 + new_x2) / 2 / w
    new_cy = (new_y1 + new_y2) / 2 / h
    new_bw = (new_x2 - new_x1) / w
    new_bh = (new_y2 - new_y1) / h

    return image, (class_id, new_cx, new_cy, new_bw, new_bh)

def process_folder(images_folder, annotations_folder, output_folder, scale_factor=1.5):
    """
    Processes a folder of images and annotations, scaling objects in images and updating annotations.
    Args:
        images_folder (str): Path to the folder containing images.
        annotations_folder (str): Path to the folder containing YOLO annotations.
        output_folder (str): Path to the output folder.
        scale_factor (float): Factor by which to scale objects.
    """
    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(os.path.join(output_folder, "images"), exist_ok=True)
    os.makedirs(os.path.join(output_folder, "annotations"), exist_ok=True)

    for image_file in os.listdir(images_folder):
        if not image_file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            continue
        
        base_name = os.path.splitext(image_file)[0]
        annotation_file = os.path.join(annotations_folder, f"{base_name}.txt")
        if not os.path.exists(annotation_file):
            continue

        # Load image and annotation
        image_path = os.path.join(images_folder, image_file)
        image = cv2.imread(image_path)
        h, w, _ = image.shape

        with open(annotation_file, "r") as f:
            annotations = [list(map(float, line.strip().split())) for line in f.readlines()]

        new_annotations = []
        for bbox in annotations:
            image, new_bbox = scale_object(image, bbox, scale_factor)
            new_annotations.append(new_bbox)

        # Save the updated image and annotation
        output_image_path = os.path.join(output_folder, "images", image_file)
        output_annotation_path = os.path.join(output_folder, "annotations", f"{base_name}.txt")

        cv2.imwrite(output_image_path, image)

        with open(output_annotation_path, "w") as f:
            for ann in new_annotations:
                f.write(" ".join(f"{x:.6f}" for x in ann) + "\n")

images_folder = "D:\FlagDetectionDatasets\ExportedTaskDatasetsZips\test\testimages"
annotations_folder = "D:\FlagDetectionDatasets\ExportedTaskDatasetsZips\test\testannotatioms"
output_folder = "D:\FlagDetectionDatasets\ExportedTaskDatasetsZips\Job_73\Job_73_scaled"
scale_factor = 1.5

process_folder(images_folder, annotations_folder, output_folder, scale_factor)


ValueError: could not broadcast input array from shape (1068,134,3) into shape (893,134,3)